In [2]:
import cfscrape
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv


def get_soup(url):
    scraper = cfscrape.create_scraper()
    req = scraper.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'lxml')
    return soup

In [1]:
import pandas as pd
def get_player(name):
    url = 'https://pubg.me/player/'
    link_pool = []
    for player in name:
        link = url + player + '/duo'
        link_pool.append(link)
    return link_pool

# concatenate all the data
def get_all(link_pool):
    i=0
    while(get_data(link_pool[i])[0] == ['ID']):
        i = i+1
    data_list = np.array(get_data(link_pool[i])[0])
    for link in link_pool:
        user = get_data(link)[1]
        if len(user) == 1:                          
            data_list = np.vstack([data_list, user+["NAN"]*37])
        else:
            data_list = np.vstack([data_list, user])
    return data_list


# get player data from each page
def get_data(url):
    soup = get_soup(url)
    data = soup.find_all('div', class_='card mb-3')
    ID = url.split("/")[4]
    if data == []:
        return [["ID"] + ["NAN"]*37 , [ID] + ["NAN"]*37]
    else:
        data_list = []
        for i in data:
            data_list += i.find_all('div', class_='col-md-4')
        value_list = []
        label_list = []
        label_list.append("ID")
        value_list.append(ID)
        for i in data_list:
            for j in i.contents:
                value_list.append(j.contents[0].string)
                label_list.append(j.contents[-1].string)
        return [label_list, value_list]


In [3]:
data = pd.read_csv("PUBG_Player_Statistics.csv")
name = data['player_name']

In [4]:
name = name.tolist()

In [5]:
pool = get_player(name)

In [12]:
def out_csv(pool):
    data = get_all(pool)
    name = 'pubg_data_duo.csv' 
    with open(name, "w") as f:
        writer = csv.writer(f)
        writer.writerows(data)


In [ ]:
out_csv(pool[:10000])